# Importing the required libraries

In [1]:
!pip install mediapipe
!pip install opencv-python

In [1]:
import cv2
import mediapipe as mp
import numpy as np

In [2]:
# to visualize the pose 
mp_drawing = mp.solutions.drawing_utils

# to import pose estimation model from the solutions section of mediapipe
mp_pose = mp.solutions.pose

In [3]:
# VIDEO FEED
# capture images from webcam 
cap = cv2.VideoCapture(0)
# loop to extract the images from camera 
while cap.isOpened():
  # frame stores the image from the camera
  ret,frame = cap.read()
  # shows the image 
  cv2.imshow('Video Feed',frame)
  # breaks the loop (no longer video capture)
  if cv2.waitKey(10) & 0xFF == ord('q'):
    break

# after breaking the loop release the camera
cap.release()
# after releasing the camera close the pop-up window of webcam
cv2.destroyAllWindows()


## Making Detection 

In [17]:

# capture images from webcam 
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence= 0.5 ,min_tracking_confidence= 0.5) as pose:
  # loop to extract the images from camera 
  while cap.isOpened():
    # frame stores the image from the camera
    ret,frame = cap.read()

    # recolor the image to RGB
    image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # process the image (make detection)
    results = pose.process(image)

    # recolor the image back to BGR
    image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    image.flags.writeable = True

    # render the image (1 -image , 2- coordinates of the landmarks , 3 - connection between these landmarks)
    mp_drawing.draw_landmarks(image, results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                              # drawing specification(joint) - color , thickness and radius of circle
                              mp_drawing.DrawingSpec(color = (245,117,66) ,thickness = 2 ,circle_radius= 2),
                              # drawin specification(connection) - color , thickness and radius of circle
                             mp_drawing.DrawingSpec(color = (245,66,230) , thickness = 2, circle_radius =2)
                              )


    # shows the image 
    cv2.imshow('Video Feed',image)
    # breaks the loop (no longer video capture): press q to close the feed 
    if cv2.waitKey(10) & 0xFF == ord('q'):
      break

  # after breaking the loop release the camera
  cap.release()
  # after releasing the camera close the pop-up window of webcam
  cv2.destroyAllWindows()



## Determining Joints

In [32]:

# capture images from webcam 
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence= 0.5 ,min_tracking_confidence= 0.5) as pose:
  # loop to extract the images from camera 
  while cap.isOpened():
    # frame stores the image from the camera
    ret,frame = cap.read()

    # recolor the image to RGB
    image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # process the image (make detection)
    results = pose.process(image)

    # recolor the image back to BGR
    image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    image.flags.writeable = True

    # extract the landmarks - we use try & except because the cam. might not always capture the landmarks 
    try:
      landmarks = results.pose_landmarks.landmark
    except:
      pass

    # render the image (1 -image , 2- coordinates of the landmarks , 3 - connection between these landmarks)
    mp_drawing.draw_landmarks(image, results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                              # drawing specification(joint) - color , thickness and radius of circle
                              mp_drawing.DrawingSpec(color = (245,117,66) ,thickness = 2 ,circle_radius= 2),
                              # drawin specification(connection) - color , thickness and radius of circle
                              mp_drawing.DrawingSpec(color = (245,66,230) , thickness = 2, circle_radius =2)
                              )


    # shows the image 
    cv2.imshow('Video Feed',image)
    # breaks the loop (no longer video capture): press q to close the feed 
    if cv2.waitKey(10) & 0xFF == ord('q'):
      break

  # after breaking the loop release the camera
  cap.release()
  # after releasing the camera close the pop-up window of webcam
  cv2.destroyAllWindows()



In [38]:
mp_pose.PoseLandmark

<enum 'PoseLandmark'>

In [39]:
for i in mp_pose.PoseLandmark:
  print(i)

PoseLandmark.NOSE
PoseLandmark.LEFT_EYE_INNER
PoseLandmark.LEFT_EYE
PoseLandmark.LEFT_EYE_OUTER
PoseLandmark.RIGHT_EYE_INNER
PoseLandmark.RIGHT_EYE
PoseLandmark.RIGHT_EYE_OUTER
PoseLandmark.LEFT_EAR
PoseLandmark.RIGHT_EAR
PoseLandmark.MOUTH_LEFT
PoseLandmark.MOUTH_RIGHT
PoseLandmark.LEFT_SHOULDER
PoseLandmark.RIGHT_SHOULDER
PoseLandmark.LEFT_ELBOW
PoseLandmark.RIGHT_ELBOW
PoseLandmark.LEFT_WRIST
PoseLandmark.RIGHT_WRIST
PoseLandmark.LEFT_PINKY
PoseLandmark.RIGHT_PINKY
PoseLandmark.LEFT_INDEX
PoseLandmark.RIGHT_INDEX
PoseLandmark.LEFT_THUMB
PoseLandmark.RIGHT_THUMB
PoseLandmark.LEFT_HIP
PoseLandmark.RIGHT_HIP
PoseLandmark.LEFT_KNEE
PoseLandmark.RIGHT_KNEE
PoseLandmark.LEFT_ANKLE
PoseLandmark.RIGHT_ANKLE
PoseLandmark.LEFT_HEEL
PoseLandmark.RIGHT_HEEL
PoseLandmark.LEFT_FOOT_INDEX
PoseLandmark.RIGHT_FOOT_INDEX


In [40]:
len(landmarks)

33

In [42]:
# this acts as an index
mp_pose.PoseLandmark.LEFT_WRIST.value

15

In [41]:
# landmarks is containing the processed image with landmark coordinates
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

x: 0.93498677
y: 1.8053514
z: -1.0772727
visibility: 0.035172947

In [43]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

x: 0.970327
y: 1.4543006
z: -0.50764984
visibility: 0.09461159

In [44]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.8031043
y: 1.0015548
z: -0.4280127
visibility: 0.9920565

## Calculating Angle

In [3]:
def calculate_angle(a,b,c):
#     a ,b,c - to store the x,y coord. of 3 landmarks(left shoulder, elbow,wrist)
    a= np.array(a) #First
    b= np.array(b) #Mid (angle will show up in the mid point)
    c= np.array(c) #End
    
#   calc. the radians between these 3 landmarks
    radians = np.arctan2(c[1] - b[1] , c[0] - b[0] ) - np.arctan2(a[1] - b[1],a[0] - b[0])
#   converting it radian value to angle
    angle = np.abs(radians*180.0/np.pi )
#   angle cannot be greater than 180 so..
    if angle>180.0:
        angle = 360.0 - angle
    
    return angle
    

In [4]:
def calculate_angle(a,b,c):
#     a ,b,c - to store the x,y coord. of 3 landmarks(left shoulder, elbow,wrist)
    a= np.array(a) #First
    b= np.array(b) #Mid (angle will show up in the mid point)
    c= np.array(c) #End
    
#   calc. the radians between these 3 landmarks
    radians = np.arctan2(c[1] - b[1] , c[0] - b[0] ) - np.arctan2(a[1] - b[1],a[0] - b[0])
#   converting it radian value to angle
    angle2 = np.abs(radians*180.0/np.pi )
#   angle cannot be greater than 180 so..
    if angle2>180.0:
        angle2 = 360.0 - angle2
    
    return angle2
    

In [5]:
calculate_angle(shoulder1 ,elbow1,wrist1)


NameError: name 'shoulder1' is not defined

In [11]:

# capture images from webcam 
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence= 0.5 ,min_tracking_confidence= 0.5) as pose:
  # loop to extract the images from camera 
  while cap.isOpened():
    # frame stores the image from the camera
    ret,frame = cap.read()

    # recolor the image to RGB
    image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # process the image (make detection)
    results = pose.process(image)

    # recolor the image back to BGR
#     image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
#     image.flags.writeable = True

    # extract the landmarks - we use try & except because the cam. might not always capture the landmarks 
    try:
      landmarks = results.pose_landmarks.landmark
#     get coordinates of the landmarks 
      shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x , landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
      elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x , landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
      wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x , landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
#     calculate angle
      angle = calculate_angle(shoulder,elbow,wrist)
#     Visualize angle
      cv2.putText(image, str(angle), 
                 tuple(np.multiply(elbow, [640,480]).astype(int)), 
                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2,cv2.LINE_AA)
    
      cv2.putText(image, str(angle2), 
                 tuple(np.multiply(elbow, [640,480]).astype(int)), 
                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2,cv2.LINE_AA)
    except:
      pass

    # render the image (1 -image , 2- coordinates of the landmarks , 3 - connection between these landmarks)
    mp_drawing.draw_landmarks(image, results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                              # drawing specification(joint) - color , thickness and radius of circle
                              mp_drawing.DrawingSpec(color = (245,117,66) ,thickness = 2 ,circle_radius= 2),
                              # drawin specification(connection) - color , thickness and radius of circle
                              mp_drawing.DrawingSpec(color = (245,66,230) , thickness = 2, circle_radius =2)
                              )


    # shows the image 
    cv2.imshow('Video Feed',image)
    # breaks the loop (no longer video capture): press q to close the feed 
    if cv2.waitKey(10) & 0xFF == ord('q'):
      break

  # after breaking the loop release the camera
  cap.release()
  # after releasing the camera close the pop-up window of webcam
  cv2.destroyAllWindows()

## Curl Counter

In [7]:
 # setting up counter 
counter = 0
counter1=0
# stage up or down
stage = None
stage1= None

# capture images from webcam 
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence= 0.5 ,min_tracking_confidence= 0.5) as pose:
  # loop to extract the images from camera 
  while cap.isOpened():
    # frame stores the image from the camera
    ret,frame = cap.read()

    # recolor the image to RGB
    image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # process the image (make detection)
    results = pose.process(image)

    # recolor the image back to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)



    # extract the landmarks - we use try & except because the cam. might not always capture the landmarks 
    try:
      landmarks = results.pose_landmarks.landmark
#     get coordinates of the landmarks 
      shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x , landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
      elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x , landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
      wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x , landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
      shoulder1 = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x , landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
      elbow1 = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x , landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
      wrist1 = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x , landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
#     calculate angle
      angle = calculate_angle(shoulder,elbow,wrist)
      angle2 = calculate_angle(shoulder1,elbow1,wrist1)
#     Visualize angle
      cv2.putText(image, str(angle), 
                 tuple(np.multiply(elbow, [640,480]).astype(int)), 
                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2,cv2.LINE_AA)
      cv2.putText(image, str(angle2), 
                 tuple(np.multiply(elbow1, [640,480]).astype(int)), 
                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2,cv2.LINE_AA)
#     curl counter logic  
      if angle > 160 and angle2 >160:
            stage ='down'
            
      if angle <30 and angle2 <30 and stage == 'down':
        stage ='all up'
        counter+=1
        print(counter)
      
      if angle < 30 and angle2 >160 and stage == 'up':
            stage ='left up'
            counter+=1
            print(counter)
            
      if angle2 <30 and angle >160 and stage == 'up':
        stage = 'right up'
        counter+=1
        print(counter)
        
      if angle < 30 and angle2 >160 and stage == 'down':
            stage ='left up'
            counter+=1
            print(counter)
      if angle2 <30 and angle >160 and stage == 'down':
        stage = 'right up'
        counter+=1
        print(counter)
        

    except:
      pass

#     render curl counter
#     displaying the status of counter 
    cv2.rectangle(image,(0,0),(300,100),(245,117,16),-1)
#     putting counter text in to the status box
#   Rep data
    cv2.putText(image,'REPS',(15,15),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0) ,1, cv2.LINE_AA)
    cv2.putText(image,str(counter),(10,80),
               cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255) ,3, cv2.LINE_AA)

#     Stage data
    cv2.putText(image,'Stage: ',(100,18),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0) ,1, cv2.LINE_AA)
    cv2.putText(image,stage,(150,20),
               cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255) ,2, cv2.LINE_AA)


    # render the image (1 -image , 2- coordinates of the landmarks , 3 - connection between these landmarks)
    mp_drawing.draw_landmarks(image, results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                              # drawing specification(joint) - color , thickness and radius of circle
                              mp_drawing.DrawingSpec(color = (245,117,66) ,thickness = 2 ,circle_radius= 2),
                              # drawin specification(connection) - color , thickness and radius of circle
                              mp_drawing.DrawingSpec(color = (245,66,230) , thickness = 2, circle_radius =2)
                              )


    # shows the image 
    cv2.imshow('Video Feed',image)
    # breaks the loop (no longer video capture): press q to close the feed 
    if cv2.waitKey(10) & 0xFF == ord('q'):
      break

  # after breaking the loop release the camera
  cap.release()
  # after releasing the camera close the pop-up window of webcam
  cv2.destroyAllWindows()

1
2
